In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("video recommendation").config("spark.some.config.option", "some-value").getOrCreate()

In [2]:
inputPath="/home/jatin/Documents/spark-notebook-0.6.2-scala-2.11.7-spark-1.6.0-hadoop-2.2.0-with-hive-with-parquet/notebooks/video-recommendation/videoMetaDataInCsvFormat"

In [5]:
englishStopWords = ["a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the"]

In [6]:
#Reading file containing video meta data to make initial RDD
readFile = sc.textFile(inputPath).cache()
readFile.count()

59800

In [7]:
readFile.take(5)

[u'0_04xqwyas,\'Edge of Alaska\': Keeping a Frontier Town Remote,All it takes is one big attraction to change everything. Will McCarthy be next?,"reality tv series, edge of alaska, mccarthy alaska, frontier town, off grid living, takeetna alaska"',
 u'0_0m8u8g4c,\'MythBusters\': Shark Feeding Frenzy Defense,"While testing the repelling abilities of ""essence of dead shark"", Jamie hopes his beret will serve as more than just a fashion statement.","reality tv series, mythbusters, jamie hyneman, adam savage, urban legends, shark myths, feeding frenzy, mythbusters shark tests, shark armor"',
 u'0_0pd5gia9,\'MythBusters\': A MythBusting Lightsaber Duel,Jamie and Adam put the myth of higher ground to the test in this epic lightsaber battle.,"reality tv series, mythbusters, jamie hyneman, adam savage, urban legends, lightsaber dueling"',
 u'0_0uwz3tkl,Behind the Scenes - Lucky Influencers: What Was Your Fashion First?,"5 fashion industry influencers, Va$shtie Kola, Lily Kwong, Amirah Kassem,

In [8]:
def formatInput(item):
     name = item.split(",")[0]
     return (name,item.replace(name+",",""))

nameFromCsv = readFile.map(formatInput)
#                                name = item.split(",")(0)
#                               (name,item.replace(name+",",""))
#                               }

In [9]:
nameFromCsv.take(2)

[(u'0_04xqwyas',
  u'\'Edge of Alaska\': Keeping a Frontier Town Remote,All it takes is one big attraction to change everything. Will McCarthy be next?,"reality tv series, edge of alaska, mccarthy alaska, frontier town, off grid living, takeetna alaska"'),
 (u'0_0m8u8g4c',
  u'\'MythBusters\': Shark Feeding Frenzy Defense,"While testing the repelling abilities of ""essence of dead shark"", Jamie hopes his beret will serve as more than just a fashion statement.","reality tv series, mythbusters, jamie hyneman, adam savage, urban legends, shark myths, feeding frenzy, mythbusters shark tests, shark armor"')]

In [10]:
reserveWords = sc.broadcast(englishStopWords)

In [11]:
import re
def bagOfWords(item):
    regexSpecialChars = """[^a-zA-Z0-9\s]"""
    noSymbolsLine = re.sub(regexSpecialChars, " ",item)
    stopWords = reserveWords.value
    listOfWords = noSymbolsLine.split()
    nonEmptyWords = filter(lambda x :x != "",listOfWords)
    removeReserveWords = []
    for word in nonEmptyWords:
        if word not in stopWords:
            removeReserveWords.append(word.lower())
    return removeReserveWords

In [12]:
bow = bagOfWords("'Edge of Alaska': Keeping a Frontier 1232 Town  @ #Remote,All it takes is () *%* one big attraction to change everything. Will McCarthy be next?,\"reality tv series, edge of alaska, mccarthy alaska, frontier town, off grid living, takeetna alaska\"")
print bow

['edge', 'alaska', 'keeping', 'frontier', '1232', 'town', 'remote', 'all', 'takes', 'big', 'attraction', 'change', 'will', 'mccarthy', 'reality', 'tv', 'series', 'edge', 'alaska', 'mccarthy', 'alaska', 'frontier', 'town', 'grid', 'living', 'takeetna', 'alaska']


In [13]:
cleandData = nameFromCsv.mapValues(bagOfWords).cache()
print cleandData

PythonRDD[8] at RDD at PythonRDD.scala:48


In [14]:
# Cross Checking count of data after cleaning 
numDocs = cleandData.count()

In [15]:
print numDocs

59800


In [16]:
readFile.unpersist()
from collections import Counter
def calculateTermFreq(item):
    return Counter(item).items()

In [17]:
it = calculateTermFreq(bow)

In [18]:
print it

[('town', 2), ('living', 1), ('keeping', 1), ('remote', 1), ('takes', 1), ('1232', 1), ('frontier', 2), ('tv', 1), ('big', 1), ('alaska', 4), ('mccarthy', 2), ('will', 1), ('attraction', 1), ('edge', 2), ('grid', 1), ('all', 1), ('series', 1), ('takeetna', 1), ('reality', 1), ('change', 1)]


In [19]:
termDocumentFrequencies = cleandData.mapValues(calculateTermFreq)

In [20]:
sampleItem = termDocumentFrequencies.take(1)[0][1]
print sampleItem

[(u'town', 2), (u'living', 1), (u'keeping', 1), (u'remote', 1), (u'takes', 1), (u'frontier', 2), (u'tv', 1), (u'big', 1), (u'alaska', 4), (u'mccarthy', 2), (u'will', 1), (u'attraction', 1), (u'edge', 2), (u'grid', 1), (u'all', 1), (u'series', 1), (u'takeetna', 1), (u'reality', 1), (u'change', 1)]


In [21]:
idDocs = termDocumentFrequencies.map(lambda x: x[0]).zipWithUniqueId().collectAsMap()#.toMap

In [22]:
type(idDocs)

dict

In [23]:
docIds = dict((v,k) for k,v in idDocs.iteritems())
# docIds.take(2)

In [24]:
docIds[0]

u'0_04xqwyas'

In [25]:
docVect = termDocumentFrequencies.flatMapValues(lambda x:x).values().reduceByKey(lambda x,y:x + y).cache()

In [26]:
cleandData.unpersist()

PythonRDD[8] at RDD at PythonRDD.scala:48

In [27]:
docVect.count()

34629

In [28]:
docVect.take(10)[2][0]

u'abenteuerliche'

In [29]:
vocabRdd = docVect.filter(lambda item: item[1] > 5 and len(item[0]) > 3).cache()
corpusVocab = vocabRdd.collect()

In [30]:
numOfFeatures = vocabRdd.count()
vocabRdd.unpersist()
print numOfFeatures

9834


In [31]:
# FUNCTION TO COMPUTE INVERSE DCOUMENT FREQUENCIES
from math import log1p
def inverseDocumentFrequencies(docFreqs, numDocs):
    return map( lambda tup : (tup[0], log1p(float(numDocs) / tup[1])), docFreqs)

In [32]:
#Computing inverse document frequencies 'idfs' from document frequencies
idfs = inverseDocumentFrequencies(corpusVocab, numDocs)

In [33]:
idfsMap = dict(idfs)

In [34]:
# /* Broadcasting 'idfs' across nodes of cluster*/
bidfs = sc.broadcast(idfsMap)

In [72]:
# /* Collecting all the terms after filtering (terms, df) pairs*/
termsIds = dict(enumerate(idfsMap.keys()))#.zipWithIndex()
# print termsIds/

In [74]:
type(termsIds)

dict

In [36]:
vocabulary = dict((v,k) for k,v in termsIds.iteritems())
termList = sc.broadcast(vocabulary)

In [37]:
docVectWithId =  docVect.zipWithIndex().map(lambda x:(x[1],x[0])).cache()
# // val pairs = docVectWithId.values.keys
docVect.unpersist()

PythonRDD[16] at RDD at PythonRDD.scala:48

In [38]:
import numpy
from pyspark.mllib.linalg import *
from pyspark import StorageLevel
from pyspark.mllib.linalg.distributed import RowMatrix
def createVector(item):
    itemDict = dict(item)
    allIdentifiers = termList.value#/* Locally obtaining broadcasted  values */
    termInThisDocument = itemDict.keys()#/* Obtaining all terms from this document*/
    sizeOfVector = len(allIdentifiers)#/* Computing number of terms(identifiers) across all the documents*/
    tfidfMap = dict()#/* Computing a map of (identifier, tfidf) pairs from term-document
#            (identifier, count) pairs and document-frequency (identifier, idfs) pair */
    for term in termInThisDocument:
        if term in allIdentifiers:
            tfidfMap[allIdentifiers[term]]=itemDict[term]
    return Vectors.sparse(numOfFeatures,tfidfMap)
#     return tfidfMap
tfidfVector = termDocumentFrequencies.mapValues(createVector)
# import org.apache.spark.mllib.linalg.{Matrix, SingularValueDecomposition, Vectors, Matrices, Vector}
# import org.apache.spark.storage.StorageLevel

# /* Constructing vector for metData of every video */

# val tfidfVector: RDD[(String, Vector)] = termDocumentFrequencies.mapValues{termFreqPair =>
#       val allIdentifiers = termList.value/* Locally obtaining broadcasted  values */
#       //val docTotalTerms: Double = termFreqPair.values.sum + 0.0
#       val termInThisDocument: List[String] = termFreqPair.keySet.toList/* Obtaining all terms from this document*/
#       val sizeOfVector: Int = allIdentifiers.size/* Computing number of terms(identifiers) across all the documents*/
#       var tfidfMap:Map[Int,Double] = Map()/* Computing a map of (identifier, tfidf) pairs from term-document
#            (identifier, count) pairs and document-frequency (identifier, idfs) pair */
#       for(term <- termInThisDocument if allIdentifiers.contains(term)) {
#         tfidfMap += (allIdentifiers(term) -> termFreqPair(term)) /* TFIDF computation */
#       }
#       val tfidfSeq: Seq[(Int, Double)] = tfidfMap.toSeq/* Converting 'tfidfMap' map to a sequence */
#       Vectors.sparse(sizeOfVector, tfidfSeq) /*Obtaining sparse vector from 'tfidfSeq' sequence and 'sizeOfVector'*/
# }
# tfidfVector.persist(StorageLevel.MEMORY_AND_DISK)

In [39]:
type(tfidfVector)

pyspark.rdd.PipelinedRDD

In [40]:
createVector(sampleItem)

SparseVector(9834, {184: 1.0, 765: 1.0, 1299: 1.0, 2943: 1.0, 3044: 1.0, 3696: 1.0, 5061: 2.0, 5719: 1.0, 6034: 1.0, 6154: 1.0, 6259: 2.0, 6828: 4.0, 9086: 2.0, 9439: 2.0, 9633: 1.0})

In [41]:
createVector(sampleItem)

SparseVector(9834, {184: 1.0, 765: 1.0, 1299: 1.0, 2943: 1.0, 3044: 1.0, 3696: 1.0, 5061: 2.0, 5719: 1.0, 6034: 1.0, 6154: 1.0, 6259: 2.0, 6828: 4.0, 9086: 2.0, 9439: 2.0, 9633: 1.0})

In [42]:
tfidfVector.persist(StorageLevel.MEMORY_AND_DISK)
tfidfVector.count()
docVect.unpersist()

PythonRDD[16] at RDD at PythonRDD.scala:48

In [43]:
#/* Constructing row matrix for terms and metaData of each video */
mat = RowMatrix(tfidfVector.values())
m = mat.numRows()# /* Number of rows in a matrix */
n= mat.numCols()# /* Number of columns in a matrix */
#/* Computing svd from the 'mat' to obtain matrices*/
# svd = mat.computeSVD(30, computeU=true)

In [42]:
samMat = tfidfVector.values().map(lambda item: item.toArray())

In [43]:
# samMat.saveAsTextFile("matrix")
type(samMat)

pyspark.rdd.PipelinedRDD

In [56]:
# sm = samMat.collect()

In [ ]:
type(sm)

In [ ]:
type(samMat.first())

In [ ]:
import numpy as np

In [ ]:
a = np.random.randn(9, 6) + 1j*np.random.randn(9, 6)

In [ ]:
type(a)

In [ ]:
# U, s, V = np.linalg.svd(samMat.collect(), full_matrices=True)

In [ ]:
# type(U)

In [ ]:
U.shape, V.shape, s.shape

In [ ]:
S = np.zeros((9, 6), dtype=complex)

In [ ]:
S[:6, :6] = np.diag(s)

In [ ]:
np.allclose(a, np.dot(U, np.dot(S, V)))

In [ ]:
U, s, V = np.linalg.svd(mat, full_matrices=True)

In [ ]:
import numpy, scipy.sparse

In [ ]:
from sparsesvd import sparsesvd

In [ ]:
mat = numpy.random.rand(200, 100)

In [ ]:
type(mat)

In [ ]:
mat.shape

In [ ]:
smat = scipy.sparse.csc_matrix(samMat)

In [ ]:
type(smat)

In [ ]:
ut, s, vt = sparsesvd(smat, 100) # do SVD, asking for 100 factors

In [ ]:
assert numpy.allclose(mat, numpy.dot(ut.T, numpy.dot(numpy.diag(s), vt)))

In [ ]:
# import genism
from gensim import corpora, models, similarities

In [44]:
# http://stackoverflow.com/questions/33428589/pyspark-and-pca-how-can-i-extract-the-eigenvectors-of-this-pca-how-can-i-calcu/33500704#33500704
from pyspark.mllib.common import callMLlibFunc, JavaModelWrapper
from pyspark.mllib.linalg.distributed import RowMatrix

In [45]:
class SVD(JavaModelWrapper):
    """Wrapper around the SVD scala case class"""
    @property
    def U(self):
        """ Returns a RowMatrix whose columns are the left singular vectors of the SVD if computeU was set to be True."""
        u = self.call("U")
        if u is not None:
            return RowMatrix(u)

    @property
    def s(self):
        """Returns a DenseVector with singular values in descending order."""
        return self.call("s")

    @property
    def V(self):
        """ Returns a DenseMatrix whose columns are the right singular vectors of the SVD."""
        return self.call("V")

In [46]:
def computeSVD(row_matrix, k, computeU=False, rCond=1e-9):
    """
    Computes the singular value decomposition of the RowMatrix.
    The given row matrix A of dimension (m X n) is decomposed into U * s * V'T where
    * s: DenseVector consisting of square root of the eigenvalues (singular values) in descending order.
    * U: (m X k) (left singular vectors) is a RowMatrix whose columns are the eigenvectors of (A X A')
    * v: (n X k) (right singular vectors) is a Matrix whose columns are the eigenvectors of (A' X A)
    :param k: number of singular values to keep. We might return less than k if there are numerically zero singular values.
    :param computeU: Whether of not to compute U. If set to be True, then U is computed by A * V * sigma^-1
    :param rCond: the reciprocal condition number. All singular values smaller than rCond * sigma(0) are treated as zero, where sigma(0) is the largest singular value.
    :returns: SVD object
    """
    java_model = row_matrix._java_matrix_wrapper.call("computeSVD", int(k), computeU, float(rCond))
    return SVD(java_model)

In [47]:
from pyspark.ml.feature import *
from pyspark.mllib.linalg import Vectors

In [48]:
data = [(Vectors.dense([0.0, 1.0, 0.0, 7.0, 0.0]),), (Vectors.dense([2.0, 0.0, 3.0, 4.0, 5.0]),), (Vectors.dense([4.0, 0.0, 0.0, 6.0, 7.0]),)]
# df = sqlContext.createDataFrame(data,["features"])

In [59]:
svd = computeSVD(mat,30,True)

In [60]:
U = svd.U

In [61]:
type(U)

pyspark.mllib.linalg.distributed.RowMatrix

In [62]:
V = svd.V

In [63]:
type(V)

pyspark.mllib.linalg.DenseMatrix

In [70]:
varr = V.toArray

In [68]:
V.numRows

9834

In [64]:
sigma = svd.s

In [65]:
type(sigma)

pyspark.mllib.linalg.DenseVector

In [66]:
print sigma

[261.340500239,157.326732017,136.585630796,121.555643588,120.213496074,116.853580174,116.185708358,114.679599252,111.598589588,110.232222097,109.363381113,107.785967392,107.209801091,106.356741268,105.83567632,105.551323328,104.611021707,103.9403295,103.289137349,102.648107231,101.871406226,101.674952761,101.120055175,100.336426178,99.4333697057,99.4116321977,98.0105621693,97.2515265761,97.0227908872,96.1518359425]


In [ ]:
def topTermsInTopConcepts(svd,numConcepts,numTerms,termIds):
    v = svd.V #Matrix representing term space
    topTerms = []
    arr = v.toArray
    rows = v.numRows
    for i in xrange(numConcepts):
        offs = i*rows
        termWeights = arr[offs:offs+rows].zipWithIndex()
        sortedArr = list(enumerate(termWeights))
        topTerms = 
# import scala.collection.mutable.ArrayBuffer

# /* FUNCTION TO COMPUTE TOP TERMS IN TOP CONCEPTS*/
#   def topTermsInTopConcepts(svd: SingularValueDecomposition[RowMatrix, Matrix], numConcepts: Int,
#       numTerms: Int, termIds: Map[Int, String]): Seq[Seq[(String, Double)]] = {
#     val v: Matrix = svd.V /* Matrix representing term space*/
#     val topTerms: ArrayBuffer[Seq[(String, Double)]] = new ArrayBuffer[Seq[(String, Double)]]()
#     val arr: Array[Double] = v.toArray
#     for (i <- 0 until numConcepts) {
#       val offs: Int = i * v.numRows
#       val termWeights: Array[(Double, Int)] = arr.slice(offs, offs + v.numRows).zipWithIndex /* Picking each column of the matrix 'v'*/
#       val sorted: Array[(Double, Int)] = termWeights.sortBy(-_._1)
#       topTerms += sorted.take(numTerms).map{case (score, id) => (termIds(id), score)} /* Associating scores with
#       corresponding terms using termIds*/
#     }
#     topTerms
#   }

#   /* FUNCTION TO COMPUTE TOP DOCUMENTS IN TOP CONCEPTS*/
#   def topDocsInTopConcepts(svd: SingularValueDecomposition[RowMatrix, Matrix], numConcepts: Int,
#       numDocs: Int, docIds: Map[Long, String]): Seq[Seq[(String, Double)]] = {
#     val u: RowMatrix = svd.U /* Matrix representing document space*/
#     val topDocs: ArrayBuffer[Seq[(String, Double)]] = new ArrayBuffer[Seq[(String, Double)]]()
#     for (i <- 0 until numConcepts) {
#       val docWeights: RDD[(Double, Long)] = u.rows.map(_.toArray(i)).zipWithUniqueId  /* Picking each row of the row matrix 'u'*/
#       topDocs += docWeights.top(numDocs).map{case (score, id) => 
#                                              var docID = ""
#                                              try{
#                                                docID = docIds(id)
#                                              }
#                                              catch {
#                                                case ex: Exception => log.error("Doc ID does not Exist", ex)
#                                                docID = ""
#                                              }  
#                                              (docID, score)} /* Associating scores with
#       corresponding documents using docIds */
#     }
#     topDocs
#   }